### --- Day 15: Warehouse Woes ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2024/day/15

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2024, 15);
var (inputMapLines, inputMoveLines) = inputLines.SeparateBy(line => line is "").ToArray();
WriteLines(inputMapLines);
Console.WriteLine();
WriteLines(inputMoveLines, maxCols: 50);

Loading puzzle file: Day15.txt
Total lines: 71
Max line length: 1000

##################################################
#.#O.O...#..##O.#.O...#OO.O....O.O#....#.....#.OO#
#...O..O...O.....O...#...OOO.O..#.....OO....##O..#
#O....O........O#........O#O..O.#.O..OO.O.O.O.OO.#
#....#......O...O...O....#.O.OO.OOO.#OOOO.#...#..#

vv^>vv<<>v>^^v>><<>vv^>^><<>v^<vv>^<><><<><vv^v^>v
>vvvvv^<><v<<v^><vv^>^^<vvv<^^<v><^>>v>>v>>^vv<>>v
<^>^^v^<<><<>>^><<^><v^>^<>>^^<>^>v<>><v^v<v<^>^v<
<^><^<^<<^<><><<<>^<^<>^vvv<<^<vv<^<^<v^>^v><v>>v>
^v^>><^>^<<^<>>v^<^v^v>><^<>^<v<^v<v<>v^<><^vv<^vv


In [4]:
string[] testInputMapLines = [
    "##########",
    "#..O..O.O#",
    "#......O.#",
    "#.OO..O.O#",
    "#..O@..O.#",
    "#O#..O...#",
    "#O..O..O.#",
    "#.OO.O.OO#",
    "#....O...#",
    "##########",
];

string[] testInputMapMoves = [
    "<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^",
    "vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v",
    "><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<",
    "<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^",
    "^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><",
    "^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^",
    ">^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^",
    "<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>",
    "^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>",
    "v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^",
];

Part 1 looks fairly straightforward: for a given movement, check all boxes in the direction to be pushed and ensure there is a free space at the end. If so, we can move all the blocks. If not, then we've hit a wall and we can't move.

In [5]:
int DoWarehouse(string[] inputMap, string[] inputMoves)
{
    var grid = new CharGrid(inputMap).Clone();

    var robot = grid.Enumerate().Where(g => g.ch is ROBOT).Single().point;

    var moves = inputMoves.SelectMany(moveLine => moveLine.ToCharArray()).Select(ch => moveLookup[ch]);
    foreach (var direction in moves)
    {
        robot = Move(grid, robot, direction);
    }

    var boxCoOrds = grid.Enumerate().Where(g => g.ch is BOX).Select(g => GpsCoord(g.point)).Sum();
    return boxCoOrds;
}

Point Move(CharGrid grid, Point robot, Point direction)
{
    var movements = CheckMove(grid, robot, direction).ToArray();

    if (movements[^1] is (_, WALL))
    {
        // Wall at the end, can't push
        return robot;
    }

    // Do the move
    foreach (var (current, next) in movements.Zip(movements.Skip(1)))
    {
        grid[next.point] = current.item;
    }
    
    grid[robot] = SPACE;

    return robot + direction;
}

IEnumerable<(Point point, Char item)> CheckMove(CharGrid grid, Point start, Point direction)
{
    Point checkPoint = start;
    var item = grid[checkPoint];

    while (true)
    {
        yield return (checkPoint, item);

        if (item is WALL or SPACE)
        {
            break;
        }
        checkPoint += direction;
        item = grid[checkPoint];
    }
}

Dictionary<char, Point> moveLookup = new() 
{
    { '^', Up },
    { 'v', Down },
    { '<', Left },
    { '>', Right },
};

const char ROBOT = '@';
const char SPACE = '.';
const char WALL = '#';
const char BOX = 'O';

// The GPS coordinate of a box is equal to 100 times its distance from the top
// edge of the map plus its distance from the left edge of the map.
int GpsCoord(Point point) => point.X + 100 * point.Y;

In [6]:
// In the larger example, the sum of all boxes' GPS coordinates is 10092

var testAnswer = DoWarehouse(testInputMapLines, testInputMapMoves);
Console.WriteLine(testAnswer);

10092


In [7]:
// Predict the motion of the robot and boxes in the warehouse. After the robot
// is finished moving, what is the sum of all boxes' GPS coordinates?

var part1Answer = DoWarehouse(inputMapLines, inputMoveLines);
Console.WriteLine(part1Answer);

1442192


In [8]:
// 1442192 is correct!
Ensure(1442192, part1Answer);